In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
A1 = pd.read_csv("eventsA001_cut25seg.csv")
A2 = pd.read_csv("eventsA002_cut25seg.csv")
A3 = pd.read_csv("eventsA003_cut25seg.csv")
D1 = pd.read_csv("eventsD001_cut25seg.csv")
E1 = pd.read_csv("eventsE001_cut25seg.csv")
F1 = pd.read_csv("eventsF001_cut25seg.csv")
A1['event_class'] = "A1"
A2['event_class'] = "A2"
A3['event_class'] = "A3"
D1['event_class'] = "D1"
E1['event_class'] = "E1"
F1['event_class'] = "F1"

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

def save_correlation_matrix(X, feature_cols, filename='correlations.csv'):
    """Save correlation matrix to CSV"""
    corr_matrix = X.corr()
    corr_matrix.to_csv(filename)
    print(f"Correlation matrix saved to {filename}")
    return corr_matrix

def diagnose_data_issues(df, target_col='event_class'):
    """
    Diagnose common data issues that cause F-test warnings
    """
    print("=== DATA DIAGNOSIS ===")
    
    # Check basic info
    print(f"Dataset shape: {df.shape}")
    print(f"Target column: {target_col}")
    
    # Check target variable
    if target_col in df.columns:
        y = df[target_col]
        print(f"\nTarget variable analysis:")
        print(f"  Unique values: {len(np.unique(y))}")
        print(f"  Value counts: {pd.Series(y).value_counts().to_dict()}")
        print(f"  Data type: {y.dtype}")
    else:
        print(f"Warning: Target column '{target_col}' not found!")
        print(f"Available columns: {list(df.columns)}")
        return
    
    # Get feature columns
    feature_cols = [col for col in df.columns if col not in ['event_id', 'channel_name', 
                                                           'start_time', 'end_time', target_col]]
    X = df[feature_cols]
    
    print(f"\nFeature analysis:")
    print(f"  Number of features: {len(feature_cols)}")
    print(f"  Features with missing values: {X.isnull().sum().sum()}")
    print(f"  Features with infinite values: {np.isinf(X.select_dtypes(include=[np.number])).sum().sum()}")
    
    # Check for zero variance features
    numeric_features = X.select_dtypes(include=[np.number])
    zero_var_features = []
    for col in numeric_features.columns:
        if numeric_features[col].var() == 0 or pd.isna(numeric_features[col].var()):
            zero_var_features.append(col)
    
    print(f"  Zero variance features: {len(zero_var_features)}")
    if zero_var_features:
        print(f"    {zero_var_features[:5]}...")  # Show first 5
    
    # Check for highly correlated features
    corr_matrix = numeric_features.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr_pairs = [(col, row) for col in upper_tri.columns for row in upper_tri.index 
                       if upper_tri.loc[row, col] > 0.95]
    
    print(f"  Highly correlated feature pairs (>0.95): {len(high_corr_pairs)}")
    if high_corr_pairs:
        print(f"    Example: {high_corr_pairs[0]}")
    
    # Summary statistics
    print(f"\nFeature statistics:")
    desc = numeric_features.describe()
    print(f"  Mean of means: {desc.loc['mean'].mean():.2e}")
    print(f"  Mean of stds: {desc.loc['std'].mean():.2e}")
    print(f"  Features with std < 1e-10: {(desc.loc['std'] < 1e-10).sum()}")
    
    return {
        'zero_var_features': zero_var_features,
        'high_corr_pairs': high_corr_pairs,
        'feature_stats': desc
    }

def preprocess_for_analysis(df, target_col='event_class'):
    """
    Preprocess data to avoid F-test warnings
    """
    print("=== PREPROCESSING ===")
    
    # Make a copy
    df_clean = df.copy()
    
    # Get feature columns
    feature_cols = [col for col in df.columns if col not in ['event_id', 'channel_name', 
                                                           'start_time', 'end_time', target_col]]
    
    print(f"Starting with {len(feature_cols)} features")
    
    # 1. Handle infinite values
    for col in feature_cols:
        if df_clean[col].dtype in ['float64', 'float32', 'int64', 'int32']:
            inf_count = np.isinf(df_clean[col]).sum()
            if inf_count > 0:
                print(f"  Replacing {inf_count} infinite values in {col}")
                df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
    
    # 2. Handle missing values
    for col in feature_cols:
        if df_clean[col].isnull().sum() > 0:
            df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
    # 3. Remove zero variance features
    X_features = df_clean[feature_cols]
    
    var_threshold = VarianceThreshold(threshold=1e-12)  # Very small threshold
    X_filtered = var_threshold.fit_transform(X_features)
    valid_features = np.array(feature_cols)[var_threshold.get_support()]
    
    removed_count = len(feature_cols) - len(valid_features)
    if removed_count > 0:
        print(f"  Removed {removed_count} zero-variance features")
    
    # Update dataframe with valid features only
    other_cols = [col for col in df_clean.columns if col not in feature_cols]
    df_clean = df_clean[list(valid_features) + other_cols]
    
    print(f"Final feature count: {len(valid_features)}")
    
    return df_clean

def analyze_feature_importance(df, target_col='event_class', top_k=20):
    """
    Comprehensive feature importance analysis for seismic sensor data
    """
    
    # Separate features and target
    feature_cols = [col for col in df.columns if col not in ['event_id', 'channel_name', 
                                                           'start_time', 'end_time', target_col]]
    X = df[feature_cols]
    y = df[target_col]
    
    # Handle missing values and infinite values
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(X.median())
    
    # Remove features with zero or near-zero variance
    print("Checking for low-variance features...")
    
    # Remove features with variance below threshold
    var_threshold = VarianceThreshold(threshold=1e-10)
    X_filtered = var_threshold.fit_transform(X)
    valid_features = np.array(feature_cols)[var_threshold.get_support()]
    
    print(f"Removed {len(feature_cols) - len(valid_features)} low-variance features")
    print(f"Remaining features: {len(valid_features)}")
    
    # Update X with only valid features
    X = pd.DataFrame(X_filtered, columns=valid_features)
    feature_cols = list(valid_features)
    save_correlation_matrix(X, valid_features, 'feature_correlations.csv')
    
    # Check target variable
    print(f"Target classes: {np.unique(y)}")
    print(f"Target distribution: {pd.Series(y).value_counts().to_dict()}")
    
    # Ensure we have multiple classes
    if len(np.unique(y)) < 2:
        raise ValueError("Target variable must have at least 2 classes for classification")
    
    # Standardize features for some methods
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=feature_cols)
    
    results = {}
    
    # 1. Random Forest Feature Importance
    print("1. Computing Random Forest importance...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)
    rf_importance = pd.DataFrame({
        'feature': feature_cols,
        'rf_importance': rf.feature_importances_
    }).sort_values('rf_importance', ascending=False)
    results['random_forest'] = rf_importance
    
    # 2. Univariate Statistical Tests (F-score) - with error handling
    print("2. Computing F-score importance...")
    try:
        # Check if we have sufficient variance for F-test
        if len(np.unique(y)) >= 2 and X.shape[0] > len(np.unique(y)):
            selector_f = SelectKBest(score_func=f_classif, k='all')
            selector_f.fit(X, y)
            
            # Handle any NaN scores
            f_scores_values = selector_f.scores_
            f_scores_values = np.nan_to_num(f_scores_values, nan=0.0, posinf=0.0, neginf=0.0)
            
            f_scores = pd.DataFrame({
                'feature': feature_cols,
                'f_score': f_scores_values
            }).sort_values('f_score', ascending=False)
            results['f_score'] = f_scores
        else:
            print("Skipping F-score: insufficient data or classes")
            
    except Exception as e:
        print(f"F-score calculation failed: {e}")
        print("Continuing with other methods...")
    
    # 3. Mutual Information - with error handling
    print("3. Computing Mutual Information...")
    try:
        mi_scores = mutual_info_classif(X, y, random_state=42)
        mi_scores = np.nan_to_num(mi_scores, nan=0.0)
        
        mi_df = pd.DataFrame({
            'feature': feature_cols,
            'mutual_info': mi_scores
        }).sort_values('mutual_info', ascending=False)
        results['mutual_info'] = mi_df
        
    except Exception as e:
        print(f"Mutual Information calculation failed: {e}")
        print("Continuing with other methods...")
    
    # 4. Correlation with target (for numerical targets)
    if pd.api.types.is_numeric_dtype(y):
        print("4. Computing correlation with target...")
        correlations = X.corrwith(y).abs().sort_values(ascending=False)
        corr_df = pd.DataFrame({
            'feature': correlations.index,
            'abs_correlation': correlations.values
        })
        results['correlation'] = corr_df
    
    # 5. Ensemble ranking (average ranks across methods) - robust version
    print("5. Computing ensemble ranking...")
    ensemble_df = pd.DataFrame({'feature': feature_cols})
    
    valid_methods = 0
    for method, df_result in results.items():
        if df_result is not None and len(df_result) > 0:
            # Convert importance to ranks (1 = most important)
            df_result = df_result.copy()
            df_result['rank'] = range(1, len(df_result) + 1)
            ensemble_df = ensemble_df.merge(
                df_result[['feature', 'rank']], 
                on='feature', 
                how='left',
                suffixes=('', f'_{method}')
            )
            valid_methods += 1
    
    # Average rank across available methods
    rank_cols = [col for col in ensemble_df.columns if col.startswith('rank') and col != 'rank']
    
    if len(rank_cols) > 0:
        # Fill NaN ranks with worst rank + 1
        for col in rank_cols:
            max_rank = ensemble_df[col].max()
            ensemble_df[col] = ensemble_df[col].fillna(max_rank + 1)
        
        ensemble_df['avg_rank'] = ensemble_df[rank_cols].mean(axis=1)
        ensemble_df = ensemble_df.sort_values('avg_rank')
        results['ensemble'] = ensemble_df
        print(f"Ensemble ranking computed using {valid_methods} methods")
    else:
        print("Warning: No valid ranking methods available")
        # Fallback to random forest only
        if 'random_forest' in results:
            ensemble_df = results['random_forest'].copy()
            ensemble_df['avg_rank'] = range(1, len(ensemble_df) + 1)
            results['ensemble'] = ensemble_df
    
    # 6. Feature groups analysis (categorize by type)
    feature_groups = categorize_features(feature_cols)
    group_importance = analyze_feature_groups(results['ensemble'], feature_groups)
    results['group_analysis'] = group_importance
    
    return results

def categorize_features(feature_cols):
    """Categorize features into logical groups"""
    groups = {
        'amplitude': ['min_amplitude', 'max_amplitude', 'mean_amplitude', 'median_amplitude', 
                     'std_amplitude', 'peak_to_peak_amplitude', 'rms_amplitude'],
        'envelope': ['min_envelope', 'max_envelope', 'mean_envelope', 'median_envelope', 'std_envelope'],
        'frequency': ['peak_frequency', 'min_freq_in_time_window', 'max_freq_in_time_window',
                     'mean_freq_in_time_window', 'median_freq_in_time_window', 'std_freq_in_time_window',
                     'min_freq_weighted_by_mag', 'max_freq_weighted_by_mag', 'mean_freq_weighted_by_mag',
                     'std_freq_weighted_by_mag'],
        'phase': ['inst_phase_slope', 'inst_phase_inter'],
        'statistical': ['kurtosis', 'skewness'],
        'information': ['kullback_leibler_div', 'shannon_entropy'],
        'response': ['third_order_freq_resp_coef_a', 'third_order_freq_resp_coef_b',
                    'third_order_freq_resp_coef_c', 'third_order_freq_resp_coef_d'],
        'power': ['signal_power']
    }
    return groups

def analyze_feature_groups(ensemble_results, feature_groups):
    """Analyze importance by feature groups"""
    group_stats = {}
    
    for group_name, features in feature_groups.items():
        group_features = [f for f in features if f in ensemble_results['feature'].values]
        if group_features:
            group_ranks = ensemble_results[ensemble_results['feature'].isin(group_features)]['avg_rank']
            group_stats[group_name] = {
                'mean_rank': group_ranks.mean(),
                'min_rank': group_ranks.min(),
                'feature_count': len(group_features),
                'top_feature': ensemble_results[ensemble_results['feature'].isin(group_features)].iloc[0]['feature']
            }
    
    return pd.DataFrame(group_stats).T.sort_values('mean_rank')

def plot_feature_importance(results, top_k=15):
    """Create visualizations of feature importance"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Random Forest importance
    if 'random_forest' in results:
        top_rf = results['random_forest'].head(top_k)
        axes[0,0].barh(range(len(top_rf)), top_rf['rf_importance'])
        axes[0,0].set_yticks(range(len(top_rf)))
        axes[0,0].set_yticklabels(top_rf['feature'])
        axes[0,0].set_title('Random Forest Feature Importance')
        axes[0,0].invert_yaxis()
    
    # F-score importance
    if 'f_score' in results:
        top_f = results['f_score'].head(top_k)
        axes[0,1].barh(range(len(top_f)), top_f['f_score'])
        axes[0,1].set_yticks(range(len(top_f)))
        axes[0,1].set_yticklabels(top_f['feature'])
        axes[0,1].set_title('F-Score Feature Importance')
        axes[0,1].invert_yaxis()
    else:
        axes[0,1].text(0.5, 0.5, 'F-Score not available', ha='center', va='center')
        axes[0,1].set_title('F-Score Feature Importance')
    
    # Mutual Information
    if 'mutual_info' in results:
        top_mi = results['mutual_info'].head(top_k)
        axes[1,0].barh(range(len(top_mi)), top_mi['mutual_info'])
        axes[1,0].set_yticks(range(len(top_mi)))
        axes[1,0].set_yticklabels(top_mi['feature'])
        axes[1,0].set_title('Mutual Information')
        axes[1,0].invert_yaxis()
    else:
        axes[1,0].text(0.5, 0.5, 'Mutual Info not available', ha='center', va='center')
        axes[1,0].set_title('Mutual Information')
    
    # Ensemble ranking
    if 'ensemble' in results:
        top_ensemble = results['ensemble'].head(top_k)
        axes[1,1].barh(range(len(top_ensemble)), 1/top_ensemble['avg_rank'])  # Invert for importance
        axes[1,1].set_yticks(range(len(top_ensemble)))
        axes[1,1].set_yticklabels(top_ensemble['feature'])
        axes[1,1].set_title('Ensemble Ranking (1/avg_rank)')
        axes[1,1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    # Feature group analysis
    if 'group_analysis' in results:
        plt.figure(figsize=(10, 6))
        group_df = results['group_analysis']
        plt.barh(range(len(group_df)), 1/group_df['mean_rank'])
        plt.yticks(range(len(group_df)), group_df.index)
        plt.xlabel('Importance (1/mean_rank)')
        plt.title('Feature Group Importance')
        plt.gca().invert_yaxis()
        plt.show()

def validate_feature_selection(X, y, selected_features, cv_folds=5):
    """Validate performance with selected features vs all features"""
    
    # All features performance
    rf_all = RandomForestClassifier(n_estimators=100, random_state=42)
    scores_all = cross_val_score(rf_all, X, y, cv=cv_folds, scoring='accuracy')
    
    # Selected features performance
    rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
    scores_selected = cross_val_score(rf_selected, X[selected_features], y, cv=cv_folds, scoring='accuracy')
    
    print(f"All features ({X.shape[1]}): {scores_all.mean():.3f} ± {scores_all.std():.3f}")
    print(f"Selected features ({len(selected_features)}): {scores_selected.mean():.3f} ± {scores_selected.std():.3f}")
    
    return scores_all.mean(), scores_selected.mean()


In [ ]:
# Run analysis
df = pd.concat([A1, A2,A3,D1,E1,F1])

feature_cols = [col for col in df.columns if col not in ['event_id', 'channel_name', 
                                                           'start_time', 'end_time', 'event_class']]
# Diagnose potential issues
diagnosis = diagnose_data_issues(df, target_col='event_class')

# Clean the data
df_clean = preprocess_for_analysis(df, target_col='event_class')

df_clean.head()

# Run analysis
results = analyze_feature_importance(df_clean, target_col='event_class', top_k=20)

# Plot results
plot_feature_importance(results, top_k=15)

# Get top features for neural network
top_features = results['ensemble'].head(15)['feature'].tolist()
print("Top 15 features for neural network:")
for i, feature in enumerate(top_features, 1):
    print(f"{i:2d}. {feature}")
    
feature_cols = [col for col in df_clean.columns if col not in ['event_id', 'channel_name', 
                                                           'start_time', 'end_time', 'event_class']]

# Validate feature selection
#validate_feature_selection(df_clean[feature_cols], df_clean['event_class'], top_features)

=== DATA DIAGNOSIS ===
Dataset shape: (154451, 39)
Target column: event_class

Target variable analysis:
  Unique values: 6
  Value counts: {'A1': 54213, 'D1': 33518, 'A2': 23970, 'F1': 18279, 'E1': 13479, 'A3': 10992}
  Data type: object

Feature analysis:
  Number of features: 34
  Features with missing values: 0
  Features with infinite values: 0
  Zero variance features: 2
    ['min_freq_in_time_window', 'min_freq_weighted_by_mag']...
  Highly correlated feature pairs (>0.95): 63
    Example: ('max_amplitude', 'min_amplitude')

Feature statistics:
  Mean of means: 1.75e+01
  Mean of stds: 1.68e+01
  Features with std < 1e-10: 3
=== PREPROCESSING ===
Starting with 34 features
  Removed 4 zero-variance features
Final feature count: 30
Checking for low-variance features...
Removed 1 low-variance features
Remaining features: 29
Correlation matrix saved to feature_correlations.csv
Target classes: ['A1' 'A2' 'A3' 'D1' 'E1' 'F1']
Target distribution: {'A1': 54213, 'D1': 33518, 'A2': 23970